# Description

In [6]:
"""
Model Result Processing
Author: Liam Megraw, RIT Envirionmental Science Technician
Date last edited: 3/16/2023
ESRI ArcGIS Pro Version 2.5.2


Description:

This code processes results from the RIT-developed computer 
vision model into thresholded predictions of presence and 
absence for multiple species at three thresholds each.

Inputs:
> 1 feature class of roads within your study area
> 1 or more feature class(es) containing unthresholded CV model points

Outputs:

The final outputs are a point layer and a line feature layer.
The point layer contains computer vision model-predicted points
above a threshold for five target species. The line layer 
contains computer vision model-predicted locations *below* the 
chosen threshold for five target species, representing absences


How to Use:

Editing is necessary to assign your geodatabse location for road 
data, as well as either a shapefile/feature class for raw model 
data OR the folder of raw model data CSVs. These edits must be made 
in the 'Define Overall Parameters - editing necessary' section. 
Either way, you must name the merged point dataset 'raw_model_pred' 
and put it into your geodatabase for the code to work as written. 
If importing raw CSVs, you must uncomment the code block titled 
'Batch import CSVs into gdb' by highlighting the text and pressing 
ctrl+/ (on windows). You must also define the folder of your raw 
CSVs, and the number of them to import.

You may also change the number of species and thresholds if desired. 
Deletions from the 'species_list' can be done independently, but 
additions require adding additional entries to the 'species_fullnames' 
list and 'threshold_dict' dictionary.

Once the script is complete, you can then input the model positives 
into the prioritization code and replace the old absence lines on
ArcGIS Online.
"""

"\nModel Result Processing\nAuthor: Liam Megraw, RIT Envirionmental Science Technician\nDate last edited: 8/26/2022\nESRI ArcGIS Pro Version 2.5.2\n\n\nDescription:\n\nThis code processes results from the RIT-developed computer \nvision model into thresholded predictions of presence and \nabsence for multiple species at three thresholds each.\n\n\nOutputs:\n\nThe final outputs are a point layer and a line feature layer.\nThe point layer contains computer vision model-predicted points\nabove a threshold for five target species. \nThe line layer contains computer vision model-predicted\nlocations *below* the chosen threshold for five target species,\nrepresenting absences\n\n\nHow to Use:\n\nEditing is necessary to assign your geodatabse location for road \ndata, as well as either a shapefile/feature class for raw model \ndata OR the folder of raw model data CSVs. These edits must be made \nin the 'Define Overall Parameters - editing necessary' section. \nEither way, you must name the me

# Define workspace

In [1]:
import arcpy
from arcpy import env
arcpy.env.overwriteOutput = True
# Enter your absolute geodatabase path containing all of your features
# here
arcpy.env.workspace = r"" 

Workspace path is already your defined geodatabase


# Define Overall Parameters - editing necessary

In [3]:
roads = "Roads_2021_NY_All"
# Define your list of feature classes to process
inputFeatures = ["Hamilton_AGOL_op",] 

# # If you renamed many files "pred_number", you can comment out
# # the "inputFeatures" line above and uncomment the three lines below. 
# # Then change the numbers in "range" to be from the starting to the 
# # ending number +1
# inputFeatures = list()
# for i in range(1,30):
#     inputFeatures.append("pred_"+str(i))

# Parameters that MAY be edited if scaling to a different numbers of 
# species

# This list of species ranges from 1-k, returning n layers equal to 
# k*thresholds*operators (e.g., 30 by default with five species, 
# three thresholds, and two operators)
species_list = ["phrag","knot","wp","toh","pl"]
# List of "species" name (some are technically groups of target 
# species) for presence records; this is used when filtering in 
# ArcGIS Online
species_fullnames = {
    # These extra sinlge quotes are intentional since 
    # they are used in a field calculation
    "phrag": "'Phragmites, Unspecified'", 
    "knot": "'Knotweed, Unspecified'",
    "wp": "'Wild Parsnip'",
    "toh": "'Tree-of-Heaven (Ailanthus)'",
    "pl": "'Purple Loosestrife'"
}
# List of optimized performance criteria suffixes
op_criteria = ["_recall", "_F1", "_precision"]
# Create a dictionary of threshold values per species at each 
# optimized criteria
threshold_dict = {
    "phrag_recall": 0.07,
    "phrag_F1": 0.33,
    "phrag_prec": 0.9,
    "knot_recall": 0.03,
    "knot_F1": 0.29,
    "knot_prec": 0.8,
    "wp_recall": 0.25,
    "wp_F1": 0.49,
    "wp_prec": 0.8,
    "toh_recall": 0.09,
    "toh_F1": 0.63,
    "toh_prec": 0.9,
    "pl_recall": 0.05,
    "pl_F1": 0.44,
    "pl_prec": 0.93
}
# This parameter should not change unless you have a good reason to 
# do so. This is the maximum distance to consider model predictions 
# continuous, defined by the 95th percentile of panorama separation 
# in Broome County, NY.
line_ag_dist = "12.4 meters" 

# Import & Merge Model Predictions (Uncomment if needed)

In [1]:
# # Batch import csvs into gdb

# # Change to path of your folder with all of your model prediction 
# # CSVs
# model_pred_folder = r"" 
# # Change 'end' to the number of model prediction CSVs in the folder 
# # you want to import
# end = 41
# start = 1

# arcpy.intelligence.BatchImportData(model_pred_folder, new_path, "*.csv", 
#                                    "NO_SUBFOLDERS"
#                                   )

# model_pred_features = list()
# for x in range(start, end, 1):
#     model_pred_features.append("pred_"+x)
# print(model_pred_features)

# # Merge all raw model point datasets into one
# raw_model_pred = 'raw_model_pred'
# arcpy.management.Merge(model_pred_features, raw_model_pred)

# Threshold, Combine, and Create Presence and Absence Features from Model Predictions

In [6]:
# Threshold, Combine, and Create Presence and Absence Features 
# from Model Predictions
import os # This module is used when deleting items

selectionType = "NEW_SELECTION"
fieldName = "Common_Nam" # Name the field for common species name
op_criteria_field = 'op_criteria'

# Define to process the conditions >= threshold and < threshold
operators = [" >= ", " < "]
record_types = ["presence", "absence"]

# Set parameters for buffering to create absence lines
bufferDistance =  line_ag_dist # Defined in cell 3
sideType = "FULL"
endType = "ROUND"
# Dissolves based on date, accounting for spatially overlapping 
# but temporally distinct records
dissolveType = "LIST" 
dissolveField = ["date"]
method = "PLANAR"

# Used when deleting items
cws = arcpy.env.workspace

print("Processing features: ")
print(inputFeatures)
# Create loop for each condition (presence and absence records), 
# ultimately returning n layers for each presence and absence
for inputFeature in inputFeatures:
    
    absenceBufferList = list()
    absenceLineList = list()
    
    print("----------Running for "+inputFeature+" ----------")
    # Add fields to contain species name and optimized performance 
    # criteria
    print("Adding fields...")
    # For common name to filter on ArcGIS Online
    arcpy.management.AddField(inputFeature, fieldName, "TEXT")
    # For decision criteria to filter on ArcGIS Online
    arcpy.management.AddField(inputFeature, op_criteria_field, "TEXT")
    # Create empty lists to add names into for merging later
    mergeListPresence = list()
    mergeListAbsence = list()
    
    for i1, operator in enumerate(operators):
        print("Creating "+record_types[i1]+" records")
        # Loop through three thresholds for each species
        for species in species_list:
            print("***Processing: "+species_fullnames[species])
            # Create a list of thresholds for the species
            thresholds = [threshold_dict[species+"_recall"],
                          threshold_dict[species+"_F1"],
                          threshold_dict[species+"_prec"]
                          ]
            # Iterate processing for each of the thresholds
            for i, threshold in enumerate(thresholds):
                # Define the conditions of threshold and date range 
                # if desired, per species
                if species is "wp":
                    whereClause = ("wild_parsnip"
                                   + operators[i1]
                                   + str(threshold) 
                                   + " And (date LIKE '%-05' Or date LIKE '%-06' Or date LIKE '%-07')"
                                  )
                if species is "toh":
                    whereClause = ("tree_of_heaven_with_seeds"
                                   + operators[i1]
                                   + str(threshold) 
                                   + " And (date LIKE '%-07' Or date LIKE '%-08' Or date LIKE '%-09' Or date LIKE '%-10')"
                                  )
                if species is "pl":
                    #loosestrife may be spelled incorrectly as 
                    # 'loostrife' in the input table
                    whereClause = ("purple_loosestrife"
                                   + operators[i1]
                                   + str(threshold) 
                                   + " And (date LIKE '%-07' Or date LIKE '%-08' Or date LIKE '%-09' Or date LIKE '%-10')"
                                  ) 
                if species is "phrag":
                    whereClause = ("phra" + operators[i1] 
                                   + str(threshold))
                if species is "knot":
                    whereClause = ("knot" + operators[i1] 
                                   + str(threshold))
                else:
                    pass
                # Return species name and recall, F1, or precision 
                # suffix
                message = (species_fullnames[species]+op_criteria[i])
                print("******Running for: "+message)
                # Select only the model predictions above/below the 
                # selected thresholds then split into a layer for 
                # each species for further analysis
                sel = arcpy.management.SelectLayerByAttribute(inputFeature, selectionType, whereClause, "") 
                # Define output name, e.g., temp_presence_phrag_recall
                outputFeature = ("temp_"+record_types[i1]+"_"
                                 +species+op_criteria[i]
                                 ) 
                # Copy features to new output
                arcpy.management.CopyFeatures(sel, outputFeature)
                # Clear selection for next run
                arcpy.management.SelectLayerByAttribute(inputFeature, 
                                                        'CLEAR_SELECTION'
                                                        )
                # Presence point attribute addition
                if operator is " >= ":
                    # add feature name to a list to feed into the 
                    # merge function later
                    mergeListPresence.append(outputFeature) 
                    # Only assign species and criteria 
                    in_table = outputFeature
                    # Set field equal to species name, indicating the
                    # record type
                    arcpy.management.CalculateField(in_table, 
                                                    fieldName, 
                                                    species_fullnames[species]
                                                    ) 
                    # Set field equal to criteria name, either 
                    # 'recall', 'F1', or 'precision'. The last 
                    # argument picks the ith string in 
                    # op_criteria then selects the characters 
                    # from 1 until the end of the string  
                    arcpy.management.CalculateField(in_table, 
                                                    op_criteria_field,
                                                    ("'"+(op_criteria[i])[1:]+"'")) 
                
                # Create absence lines
                else:
                    mergeListAbsence.append(outputFeature)
                    print("******Buffering and creating absence lines for "+outputFeature)
                    outputBuffer = (outputFeature + "_NO")
                    # Buffer the feature
                    arcpy.Buffer_analysis(outputFeature, 
                                          outputBuffer, 
                                          bufferDistance, 
                                          sideType, 
                                          endType, 
                                          dissolveType, 
                                          dissolveField, 
                                          method
                                          )
                    # Add output names to list for deleting 
                    # later
                    absenceBufferList.append(outputBuffer)
                    # Add fields to add criteria and species
                    arcpy.management.AddField(outputBuffer, 
                                              fieldName, 
                                              "TEXT"
                                              )
                    arcpy.management.AddField(outputBuffer,
                                              op_criteria_field,
                                              "TEXT"
                                              )
                    arcpy.management.CalculateField(outputBuffer, 
                                                    fieldName,
                                                    species_fullnames[species]
                                                    ) 
                    arcpy.management.CalculateField(outputBuffer,
                                                    op_criteria_field,
                                                    "'"+(op_criteria[i])[1:]+"'") 
                    # Intersect each buffer with roads
                    outputRoads = (outputFeature + "_road")
                    arcpy.analysis.Intersect([outputBuffer, roads],
                                             outputRoads, 
                                             "",
                                             "",
                                             "LINE")
                    # Add output names to list for merging
                    absenceLineList.append(outputRoads)
                    # Delete the FID field
                    field = "FID_temp_absence_"+species+op_criteria[i]+"_NO"
                    arcpy.management.DeleteField(outputRoads, 
                                                 field)
    print("Presence points and absence line records created for "+inputFeature)
    
    # Create final merged features
    if " >= " in operators:
        print("Merging processed features for "+inputFeature)
        # Merge presence points together
        outputName = ("model_predicted_presences_"+inputFeature)
        arcpy.management.Merge(mergeListPresence, outputName)
        print("***Final processed presence file named " + outputName 
              + " is complete")
    # Merge absence points together
    outputName = ("model_predicted_absences_"+inputFeature)
    arcpy.management.Merge(mergeListAbsence, outputName)
    print("***Final processed presence file named " + outputName 
          + " is complete")
    # Merge absence lines together
    outputName = ("model_not_observed_"+inputFeature)
    arcpy.management.Merge(absenceLineList, outputName)
    print("***Final processed absence file named " + outputName 
          + " is complete")
    # Delete temporary files; this way is necessary to delete the 
    # feature itself and not just its contents. The directory is
    # defined above the main loop
    print("Deleting temporary files")
    # Delete unmerged presence points
    print("***Deleting presence points")
    for input in mergeListPresence:
      input_path = os.path.join(cws, input)
      if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)
    print("***Deleting absence buffers")
    # Delete temporary absence buffers
    for input in absenceBufferList:
      input_path = os.path.join(cws, input)
      if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)
    print("***Deleting absence points")
    # Delete unmerged absence points
    for input in mergeListAbsence:
      input_path = os.path.join(cws, input)
      if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)
    print("***Deleting absence lines")
    # Delete unmerged absence lines
    for input in absenceLineList:
      input_path = os.path.join(cws, input)
      if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)
    print("Temporary files deleted")
    
    print("----------Processing completed for "
          +inputFeature+" ----------")

Processing features: 
['Hamilton_AGOL_op']
----------Running for Hamilton_AGOL_op ----------
Adding fields...
Creating absence records
***Processing: 'Phragmites, Unspecified'
******Running for: 'Phragmites, Unspecified'_recall
******Buffering and creating absence lines for temp_absence_phrag_recall
******Running for: 'Phragmites, Unspecified'_F1
******Buffering and creating absence lines for temp_absence_phrag_F1
******Running for: 'Phragmites, Unspecified'_precision
******Buffering and creating absence lines for temp_absence_phrag_precision
***Processing: 'Knotweed, Unspecified'
******Running for: 'Knotweed, Unspecified'_recall
******Buffering and creating absence lines for temp_absence_knot_recall
******Running for: 'Knotweed, Unspecified'_F1
******Buffering and creating absence lines for temp_absence_knot_F1
******Running for: 'Knotweed, Unspecified'_precision
******Buffering and creating absence lines for temp_absence_knot_precision
***Processing: 'Wild Parsnip'
******Running for: